In [77]:
import torch
from torch import nn as nn
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm 

In [87]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
# Hyperparameters
block_size = 8
batch_size = 4
max_iterations = 10000
lr = 3e-4
eval_iterations = 250
dropout = 0.2

In [54]:
with open("content.txt", 'r', encoding='utf-8') as f:
    for line in f:
        text = f.read()

characters = sorted(set(text))
vocab_size = len(characters)

In [28]:
# Creating a mapping from unique characters to indices
string_to_int = { char:index for index, char in enumerate(characters)}
int_to_string = { index:char for index, char in enumerate(characters)}

# Encoding the text
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda c : ''.join([int_to_string[i] for i in c])

# Turn into tensors 
data = torch.tensor(encode(text), dtype=torch.long)


In [79]:
split = int(len(data) * 0.8) # Hyperparameter

train_data = data[:split].to(device)
test_data = data[split:].to(device)

def get_batch(split):
    data = train_data if split == "train" else test_data
    start_index = torch.randint(len(data) - block_size, (batch_size,))
    # print(start_index)
    x = torch.stack([data[idx:idx+block_size] for idx in start_index])
    y = torch.stack([data[idx+1:idx+block_size+1] for idx in start_index])
    x, y = x.to(device), y.to(device) # Make sure data is on the right device
    return x, y

x, y = get_batch("train")
print("inputs:")
print(x.shape)
print(x)
print("targets:")
print(y)


inputs:
torch.Size([4, 8])
tensor([[47, 62,  1, 48, 51, 56, 56, 61],
        [ 1, 63, 62,  1, 43, 62, 62,  1],
        [46, 47, 62,  1, 56, 51,  1, 62],
        [61, 62, 63, 46, 47, 56, 62, 47]], device='cuda:0')
targets:
tensor([[62,  1, 48, 51, 56, 56, 61,  1],
        [63, 62,  1, 43, 62, 62,  1, 44],
        [47, 62,  1, 56, 51,  1, 62, 64],
        [62, 63, 46, 47, 56, 62, 47, 56]], device='cuda:0')


In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}")

When input is tensor([22], device='cuda:0'), the target is 47
When input is tensor([22, 47], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56], device='cuda:0'), the target is 1
When input is tensor([22, 47, 56,  1], device='cuda:0'), the target is 61
When input is tensor([22, 47, 56,  1, 61], device='cuda:0'), the target is 51
When input is tensor([22, 47, 56,  1, 61, 51], device='cuda:0'), the target is 49
When input is tensor([22, 47, 56,  1, 61, 51, 49], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56,  1, 61, 51, 49, 56], device='cuda:0'), the target is 43


In [86]:
def estimate_loss(model: nn.Module):
    out = {}
    with torch.inference_mode():
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iterations)
            for k in range(eval_iterations):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
    return out 

In [61]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


    def forward(self, index, targets=None): 
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch_size, time, vocab_size
            logits = logits.view(B*T, C) # Flatten the batch and time dimensions
            targets = targets.view(B*T) # Flatten the batch and time dimensions 
            loss = F.cross_entropy(logits, targets) # Calculate the loss 

        return logits, loss

    # Generate new content
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # Make predictions
            logits, loss = self.forward(index)
            # Get the last time step
            logits = logits[:, -1, :] # (B, C)
            # Turn into probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution or take the most likely
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append to the sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [63]:
model_0 = BigramLanguageModel(vocab_size).to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model_0.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


JgFJ;VøzeeAYëëL:KmJoS6cgrp1vPéz9ü’Ö0A4q’CLDCJFfåOn6–9ADCTpyUqSm5Öpc39 :cüG41p1EtSwUzÖ–KpXFlIGr ti7VWmUüpYhLDg.mLl’Lä?…—SEHmIeW9ükVuUz9Nñu,hag
U5hl,uFIKwO30v2oNñæWYñefsF?3?…rp–LéiwjOPføoD…HacFMOæÅæ
üB8WYg:KDtrffO?4wÄNwW-Op’v,pVdëIx3øC1zj0XTp9
hL;—ø–Pä;Uns;fC0i ÄfoB–F?Äw6D
M?J,p1qM—-4Eåünmd?KRu2O-29XF
étåti:éCqÅÖjëIvd8hüctVu2Y9!öXøWYbe?Gcæu14IkñYb9vlS3e86K6Gäu2O-olSkYg7—tTuWNñRXl8D3LnJAj3!F?D,th-… jO,s;kcI
kG:—ÖjTëL4LéäågXAWYYreu,KtP!;’.MOP76R’n.Öë::kYñLXvö2!NbfPyoB p:p1kNruUAYvYr’vvd FBfStyüCcH;!


In [84]:
optimizer = torch.optim.AdamW(model_0.parameters(), lr=lr)

for iter in tqdm(range(max_iterations)):
    if iter % eval_iterations == 0:
        print(f'Step: {iter} | Loss: {loss.item():.3f}')
    # Sample a batch of data
    xb, yb = get_batch("train")

    # Calculate the loss 
    logits, loss = model_0(xb, yb)
    optimizer.zero_grad(set_to_none=True) # Clear the gradients
    loss.backward() # Calculate the gradients
    optimizer.step() # Update the weights

print(loss.item())

100%|██████████| 10000/10000 [00:16<00:00, 618.88it/s]

2.570427179336548


In [85]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model_0.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


éz!in N4e stta slåkve m. stt vlå ar s förar KzVæ
A…ær ockumigreran F0vt lillinete harnadet nhäläpp memänan unakuten jan vale fader föriollo, fövit agattt förm. Uäranandugachottaden?O
Jdedemkna, harar hat bypp ha må Nckavade jk ppran lövadördatelyn tensarödalfanevivara veko, h anajktyg h uve hoburttretrörr kunboch nocka l sen va adetti jat?
Hänken.
 se SëqKzéKld de dadäche brtt me, s fat gretet AYva män somen, hötå ly:enara, be?Dulet venoere orågin, deplilet De höracha. Å2h-K1Ise. döm å ögdere?må
